In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("db.sqlite") #local.

In [3]:
c = conn.cursor()
# c.execute('CREATE table users (NAME text, NUMBER float)')
# c.execute('insert into users values ("Jack Bauer", "555-555-5555")')
c.execute('SELECT DISTINCT NAME FROM users')
conn.commit()

# col_name_list = [tup for tup in c.description]
# print(col_name_list)
rows = c.fetchall()

for r,idx in enumerate(rows): #~Generator
    print(r,idx)

0 ('Jack Bauer',)


In [4]:
c.close()

In [5]:
c

In [6]:
#pandas supports SQL - via read_sql (Postgres,SQLlite3)

In [6]:
import pandas as pd
pd.read_sql('select * from users', conn)

,NAME,NUMBER
0,Jack Bauer,555-555-5555
1,Jack Bauer,555-555-5555
2,Jack Bauer,555-555-5555
3,Jack Bauer,555-555-5555
4,Jack Bauer,555-555-5555
5,Jack Bauer,555-555-5555
6,Jack Bauer,555-555-5555
7,Jack Bauer,555-555-5555
8,Jack Bauer,555-555-5555
9,Jack Bauer,555-555-5555


In [8]:
#compare sql vs csv for data to df performance

In [25]:
df=pd.DataFrame([x**2 for x in range(50774)])

In [ ]:
df.to_sql(name='random_dff', con=conn)
df.to_csv('random_df')

# SQL

In [38]:
%%timeit

pd.read_sql(f'select * from random_dff', con=conn)

76 ms ± 4.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
pd.read_sql(f'select * from random_dff', con=conn)

,index,0
0,0,0
1,1,1
2,2,4
3,3,9
4,4,16
...,...,...
50769,50769,2577491361
50770,50770,2577592900
50771,50771,2577694441
50772,50772,2577795984


# CSV

In [39]:
%%timeit

pd.read_csv('random_df')

20.8 ms ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### simply loading, csv is better

#### A CSV is very naive and simple. loading directly from it will be very quick. For massive database with complex structure CSV is not an option..

#### let's see how basic computation differs; especially if we only want data from a range of dates

In [52]:
%%timeit

pd.read_sql(f'select COUNT(DISTINCT "0") from random_dff', con=conn)

54.4 ms ± 5.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
%%timeit

len(pd.read_csv('random_df')['0'].unique())

18.5 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### CSV is for sequential access, ie, you start at the beginning of the file and you read each row one after the other, treating it as needed.

### SQL is for indexed access, ie, you look at an index and then you go to the row you're looking for. You can also perform a full table scan, ie not make use of any index, which makes the table essentially a bloated CSV.

In [76]:
%%timeit

pd.read_sql(f'select COUNT(DISTINCT "0") from random_dff WHERE "index" >= 34694', con=conn)

17 ms ± 2.72 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [77]:
%%timeit

x=pd.read_csv('random_df')

len(x[x['Unnamed: 0']>34694]['0'].unique())

17.2 ms ± 659 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [78]:
## starts to get faster, once index is started to be used!...if not, sql db is just a bloated version of csv